# 機能/目的
## - Assistantが提供するPre-Trained Intentsを利用するダイアログノードをワークスペースに追加する

## 0 前提作業

#### 1) 学習済みデータを取り込む
- 学習済みインテント
    - ワークスペースを作成して、一般とe-コマースのインテントを取り込んでおく
- システムエンティティ
    - 日付、時刻、数字、パーセント、通貨といったドメインに依存しないエンティティを有効にする#### 2) Watson SDKの導入

#### 2) Watson SDKの導入

In [ ]:
!pip install --upgrade "watson-developer-cloud>=1.3.0" > /dev/null 2>&1

## 1 クリデンシャル情報のセット

In [ ]:
# -*- coding: utf-8 -*-
import json
import re
import watson_developer_cloud

workspaceid = 'xxx'
versionid = '2018-02-16'

conversation = watson_developer_cloud.ConversationV1(
    username='xxx',
    password='xxx',
    version= versionid
)

## 2 初期ノード作成 (welcome, anything_else)

In [ ]:
mynode = conversation.create_dialog_node(
    workspace_id = workspaceid,
    title = "ようこそ",
    conditions = "welcome",
    output = {'text':"こんにちは、ワトソンです"},
    dialog_node = "start"
)
mynode = conversation.create_dialog_node(
    workspace_id = workspaceid,
    title = "その他",
    conditions = "anything_else",
    output = {'text':"解釈できません"},
    dialog_node = "end",
    previous_sibling = "start"
)

## 3 フォルダ作成 for Pre-Trained Intents

In [ ]:
response = conversation.list_intents(
    workspace_id = workspaceid
)

#フォルダ名の生成
myintent = ""
myintents = []
for index, intent in enumerate(response["intents"]):
    index = intent["intent"].find('_')
    if (intent["intent"][:index] != myintent):
        myintent = intent["intent"][:index]
        myintents.append(myintent)

for fname in myintents:
    conversation.create_dialog_node(
        workspace_id = workspaceid,
        title = fname,
        conditions = "true",
        dialog_node = fname,
        node_type = "folder",
        previous_sibling = "start"
    )

## 4 Pre-Trained Intents用ダイアログノードの作成

In [ ]:
for fname in myintents:
    for index, intent in enumerate(response["intents"]):
        nodename = intent["intent"]
        if nodename.find(fname) > -1:
            myparent = fname
            conversation.create_dialog_node(
                workspace_id = workspaceid,
                dialog_node = nodename,
                conditions = '#' + nodename,
                output = { 'text': "まだ学習していません"},
                title = intent["description"],
                parent = myparent
            )

## [注意] 全ノードの削除
- 必要な場合にのみ実行

In [ ]:
response = conversation.list_dialog_nodes(
    workspace_id = workspaceid
)

for node in response["dialog_nodes"]:
    if (node["type"] == "folder" or node["dialog_node"] == "end" or node["dialog_node"] == "start"):
        nodename = node["dialog_node"]
        response = conversation.delete_dialog_node(
            workspace_id = workspaceid,
            dialog_node = nodename
        )

Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.